**برای اجرای این نوتبوک فایل داده ها باید در کنار این نوتبوک قرار داده بشه**

In [1]:
pip install -q clean-text[gpl]

     |████████████████████████████████| 170 kB 12.7 MB/s 
     |████████████████████████████████| 64 kB 2.8 MB/s 
     |████████████████████████████████| 235 kB 34.4 MB/s 


In [2]:
!pip install hazm

     |████████████████████████████████| 316 kB 12.2 MB/s 
     |████████████████████████████████| 233 kB 45.1 MB/s 
     |████████████████████████████████| 1.4 MB 48.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=cd4d9cf02665f1d41dec16b3e1c96725f7e72ed484170efbf14b50afd2192a7f
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154814 sha256=d2e688fe500305e340f1f8d45fcb1602c82417acb0f0c50a95e97255163d4e81
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 9.5 MB/s 
     |████████████████████████████████| 895 kB 29.7 MB/s 
     |████████████████████████████████| 596 kB 42.7 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 19.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext.legacy.data as data
import hazm
import sys
from cleantext import clean
import re
import pandas as pd

In [6]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text
        ,fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [7]:
original = pd.read_csv('dataset.txt', encoding='utf-8', header=None, delimiter ='$', names=['person1','person2'])
original['cleaned_person1'] = original['person1'].apply(cleaning)
original['cleaned_person2'] = original['person2'].apply(cleaning)
original = original[['cleaned_person1', 'cleaned_person2']]
original.columns = ['Question', 'Answer']
original.to_csv('persian_questions_answers.csv', index=False, encoding='utf-8')

In [8]:
def tokenizer(text):
    return [tok for tok in hazm.word_tokenize(text)]
question = data.Field(lower=True, 
                      init_token="<sos>",
                      eos_token="<eos>",
                      tokenize=tokenizer)

answer = data.Field(lower=True, 
                    init_token="<sos>", 
                    eos_token="<eos>", 
                    tokenize=tokenizer)

In [9]:
datafields = {"Question":("q", question), "Answer":("a", answer)}
dataset = data.TabularDataset(path="persian_questions_answers.csv", format="csv", fields=datafields)

In [10]:
g_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**در این قسمت دیکشنری سوال و جوابها ساخته میشه**

In [11]:
question.build_vocab(dataset, min_freq=1)
answer.build_vocab(dataset, min_freq=1)
train_data, valid_data = dataset.split(split_ratio=0.75)
train_iter = data.BucketIterator(
        train_data,
        batch_size=32,
        device=g_device,
        sort_within_batch=True,
        sort_key=lambda x: len(x.q)
)
valid_iter = data.BucketIterator(
        valid_data,
        batch_size=32,
        device=g_device,
        sort_within_batch=True,
        sort_key=lambda x: len(x.q)
)

**در این قسمت معماری مدل ایجاد میشه**

In [12]:
class Transformer(nn.Module):
    def __init__(self, embedding_size, 
                 src_vocab_size,
                 trg_vocab_size,
                 src_pad_index,
                 num_heads,
                 num_encoder_layers,
                 num_decoder_layers,
                 dense_dim,
                 dropout,
                 max_len,
                ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            dense_dim,
            dropout
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_index = src_pad_index
    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_index
        return src_mask
    
    def forward(self, src, trg):
        src_seq_len, N = src.shape
        trg_seq_len, N = trg.shape
        src_positions = (
            torch.arange(0, src_seq_len).unsqueeze(1).expand(src_seq_len, N).to(device=g_device)
        )
        trg_positions = (
            torch.arange(0, trg_seq_len).unsqueeze(1).expand(trg_seq_len, N).to(device=g_device)
        )
        embed_src = self.dropout((self.src_word_embedding(src) + self.src_position_embedding(src_positions)))
        embed_trg = self.dropout((self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions)))
        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_len).to(device=g_device)
        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask = src_padding_mask,
            tgt_mask=trg_mask
        )
        out = self.fc_out(out)
        return out

**در این قسمت پارامترهای مدل مقداردهی میشه**

In [13]:
num_epochs = 125
learning_rate = 3e-4
src_vocab_size = len(question.vocab)
trg_vocab_size= len(answer.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3
dropout = 0.1
max_len = max(max([batch.a.shape[0] for batch in train_iter]), max([batch.q.shape[0] for batch in train_iter]))
dense_dim = 64
src_pad_index = question.vocab.stoi["<pad>"]

net = Transformer(embedding_size,  src_vocab_size, 
                  trg_vocab_size, src_pad_index, 
                  num_heads, num_encoder_layers,
                  num_decoder_layers, dense_dim, dropout, max_len).to(device=g_device)

trg_pad_index = answer.vocab.stoi["<pad>"]
loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_index)
opt = optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt, factor=0.1, patience=10, verbose=True
)

In [14]:
def predict(test_sentence, model):
    max_len = 100
    # sen = [w for w in tokenizer.tokenize(test_sentence)]
    sen = [w for w in hazm.word_tokenize(test_sentence)]
    sen.insert(0, question.init_token)
    sen.append(question.eos_token)
    inp_sen = [question.vocab.stoi[i] for i in sen]
    inp_sen = torch.tensor(inp_sen, dtype=torch.long).unsqueeze(1).to(device=g_device)
    outputs = [answer.vocab.stoi["<sos>"]]
    for i in range(max_len):
        trg = torch.tensor(outputs, dtype=torch.long).unsqueeze(1).to(device=g_device)
        with torch.no_grad():
            output = model(inp_sen, trg)
        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)
        if best_guess == answer.vocab.stoi["<eos>"]:
            break
    pred_sentence = [answer.vocab.itos[i] for i in outputs]
    return pred_sentence

**در این قسمت مدل آموزش داده میشه و ارزیابی میشه و بهترین مدلها روی داده های ارزیابی ذخیره میشه**

In [16]:
test_sentence = "کشور سازنده‌ی تو" 
best_loss = sys.float_info.max
for epoch in range(num_epochs):
    losses = []
    for data in train_iter:
        net.train()
        outs = net(data.q, data.a[:-1, :])
        outs = outs.reshape(-1, outs.shape[2])
        target = data.a[1:].reshape(-1)
        opt.zero_grad()
        loss = loss_fn(outs, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=1)
        opt.step()
        losses.append(loss.item())
    mean_loss = sum(losses) / len(losses)
    print(f"[Epoch {epoch}] Loss {mean_loss}")
    net.eval()
    pred_sen = predict(test_sentence, net)
    print(" ".join(pred_sen))
    valid_losses = []
    with torch.no_grad():
        for data in valid_iter:
            outs = net(data.q, data.a[:-1, :])
            outs = outs.reshape(-1, outs.shape[2])
            target = data.a[1:].reshape(-1)
            opt.zero_grad()
            loss = loss_fn(outs, target)
            valid_losses.append(loss.item())
        valid_loss = sum(valid_losses) / len(valid_losses)
        if valid_loss < best_loss:
            best_loss = valid_loss  
              
            state = {'epoch': epoch, 'best_model': net, 'optimizer': opt}
            torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')
            print(f"***** Current best checkpoint is saved. *****")
            print(f"Best valid loss: {best_loss}")
            print(f"Valid loss: {valid_loss}")
    scheduler.step(mean_loss)


[Epoch 0] Loss 2.6181348860263824
<sos> همه من خوشمزه هستند <eos>
***** Current best checkpoint is saved. *****
Best valid loss: 2.4816747903823853
Valid loss: 2.4816747903823853
[Epoch 1] Loss 2.0355038017034532
<sos> من یک ربات هستم <eos>
***** Current best checkpoint is saved. *****
Best valid loss: 2.0003129690885544
Valid loss: 2.0003129690885544
[Epoch 2] Loss 1.6148752853274346
<sos> من یک ربات هستم <eos>
[Epoch 3] Loss 1.3648272410035134
<sos> من یک ربات هستم <eos>
***** Current best checkpoint is saved. *****
Best valid loss: 1.798896387219429
Valid loss: 1.798896387219429
[Epoch 4] Loss 1.054251068085432
<sos> من یک ربات هستم <eos>
***** Current best checkpoint is saved. *****
Best valid loss: 1.74906225502491
Valid loss: 1.74906225502491
[Epoch 5] Loss 0.8270890291780233
<sos> من یک ربات هستم <eos>
[Epoch 6] Loss 0.6652787502855062
<sos> من یک ربات هستم <eos>
[Epoch 7] Loss 0.555126441642642
<sos> من یک ربات هستم <eos>
[Epoch 8] Loss 0.47926580905914307
<sos> ایران <eos>
***

در این قسمت آدرس مدل از قبل آموزش داده شده تو تابع لود قرار داده میشه و بعد از اون با بات میشه چت کرد 

In [20]:
checkpoint = torch.load('checkpoint_8.pth.tar')
net = checkpoint['best_model']
while True:
    inp_sen = input("شما:")
    if inp_sen == 'توقف':
      break
    pred = predict(inp_sen, net)
    print('بات: '+" ".join(pred[1:-1]))

شما:خوشی
بات: خدا رو شکر خوبم
شما:اهل کجایی
بات: من یک ایرانی هستم
شما:خداحافظ
بات: به سلامت برو بذار ما هم به کارمون برسیم
شما:توقف


In [146]:
!pip install regex

In [191]:
import regex

In [210]:
m = re.search('(?<=مدل )[0-9|۰-۹]+', 'مدل 8۷9 مشکی')

In [227]:
m = regex.search('[0-9|۰-۹]+', 'قبل از ۹۸')

In [228]:
 a = m.group(0)

In [229]:
from unidecode import unidecode
b = unidecode(a)

In [230]:
a

'۹۸'

In [200]:
b

'85'

In [186]:
1385-int(b)

506